In [50]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder

# dataset loading
shelter_dogs = pd.read_csv('ShelterDogs.csv')

# change the dogs age from a number into 3 categories; puppy, adult, or old
conditions = [
    (shelter_dogs['age'] < 1),
    (shelter_dogs['age'] >= 1) & (shelter_dogs['age'] <= 8),
    (shelter_dogs['age'] > 8)
]

choices = ['puppy', 'adult', 'old']

shelter_dogs['age'] = np.select(conditions, choices, default='unknown')

# replace NaN and blank columns into unknown
shelter_dogs = shelter_dogs.fillna("unknown")
shelter_dogs = shelter_dogs.replace("", "unknown")

# replace unknown in the name column to unnamed
shelter_dogs['name'] = shelter_dogs['name'].replace("unknown", "unnamed")

# attributes that the user can choose from stored in a dictionary
attributes = {
    1: "age",
    2: "sex",
    3: "breed",
    4: "color",
    5: "coat",
    6: "size",
    7: "neutered",
    8: "housebroken",
    9: "likes_people",
    10: "likes_children",
    11: "get_along_males",
    12: "get_along_females",
    13: "get_along_cats",
    14: "keep_in"
}

# showcase the possible inputs that the user can input
specific_preferences = {
    "age": ["puppy", "adult", "old"],
    "sex": ["male", "female"],
    "breed": None,  # Can take any string
    "color": ['red', 'black and white', 'saddle back', 'yellow-brown', 'black',
       'brown', 'gray and white', 'brown and white', 'tricolor', 'spotty',
       'white', 'apricot', 'black and brown', 'golden', 'striped',
       'yellow', 'wild boar', 'black and tan', 'gray and black', 'sable',
       'gray', 'red and white', 'dotted'],
    "coat": ["long", "medium", "short", "wirehaired"],
    "size": ["small", "medium", "large"],
    "neutered": ["yes", "no"],
    "housebroken": ["yes", "no"],
    "likes_people": ["yes", "no"],
    "likes_children": ["yes", "no"],
    "get_along_males": ["yes", "no"],
    "get_along_females": ["yes", "no"],
    "get_along_cats": ["yes", "no"],
    "keep_in": ["flat", "garden", "both flat and garden"]
}

# function to get user preferences
def get_user_preferences():
    print("""Welcome to PawPal Pet Adoption Center. 
We are here to help you find your new furry friend! """)

    preferences = {}
    
    while True:
        # show the user the preferences they can pick from
        print("\nHelp us understand which attribute is most important to you? (or press Enter to finish):")
       
        # loop through the preferences and print them for the user to see
        for key, value in attributes.items():
            print(f"{key}: {value}")

        # ask the user for their choice
        choice_input = input("Enter the number of your chosen attribute (or press Enter to finish): ").strip()

        # if user inputs blank then that means they are either done selecting or have no preferences
        if choice_input == "":
            break

        try:
            choice = int(choice_input)

            if choice in attributes:
                attribute = attributes[choice]

                # ask for the specific preference if available, if not then they can input a string
                if specific_preferences[attribute] is None:
                    specific_input = input(f"Enter your specific preference for {attribute}: ").strip()
                    
                # ask for a valid response from the user, if it's invalid it would ask them again for a valid response
                else:
                    valid_options = ', '.join(specific_preferences[attribute])
                    specific_input = input(f"Enter your specific preference for {attribute} ({valid_options}): ").strip().lower()
                    if specific_input not in specific_preferences[attribute]:
                        print(f"I'm sorry, I didn't understand that. Please enter one of the following: {valid_options}.")
                        continue

                preferences[attribute] = specific_input
                print(f"You have selected: {attribute} with preference: {specific_input}")
            else:
                print("I'm sorry, I didn't understand that. Please select a valid number from the list.")
        
        except ValueError:
            print("I'm sorry, I didn't understand that. Please make sure to type a correct number.")
            continue
    
    # if no preferences selected, ask if they wish to continue without preferences
    if not preferences:
        skip = input("You haven't selected any preferences. Would you like to proceed with no preferences? (yes/no): ").strip().lower()
        if skip != "yes":
            return get_user_preferences()  # Re-run the function if the user changes their mind

    return preferences


# function to recommend a dog to a user
def recommend_dog(preferences, shelter_dogs):
    # If no preferences, filter to only old dogs
    if not preferences:
        shelter_dogs = shelter_dogs[shelter_dogs['age'] == 'old'].copy()

    # fit the encoder to the dataset
    enc = OneHotEncoder(handle_unknown='ignore')
    categorical_columns = list(attributes.values())
    encoded_dogs = enc.fit_transform(shelter_dogs[categorical_columns]).toarray()

    # create a DataFrame with all attributes, filling missing ones with None
    user_pref_df = pd.DataFrame({col: [None] for col in categorical_columns})

    # fill in the user's preferences
    for col in preferences:
        user_pref_df[col] = preferences[col]

    # transform the user's preference DataFrame
    preference_vector = enc.transform(user_pref_df).toarray()[0]

    # get the cosine similarity between the user's preferences and the shelter dogs
    similarities = cosine_similarity([preference_vector], encoded_dogs)[0]

    # set the similarity scores in the DataFrame
    shelter_dogs.loc[:, 'similarity'] = similarities

    # sort the shelter dogs by similarity score
    sorted_dogs = shelter_dogs.sort_values(by='similarity', ascending=False)

    # display the closest match
    closest_match = sorted_dogs.iloc[0]
    print("\nClosest Match:")
    print("----------------------------")
    print(f"Name: {closest_match['name']}")
    print(f"Age: {closest_match['age']}")
    print(f"Sex: {closest_match['sex']}")
    print(f"Breed: {closest_match['breed']}")
    print(f"Color: {closest_match['color']}")
    print(f"Coat: {closest_match['coat']}")
    print(f"Size: {closest_match['size']}")
    print(f"Neutered: {closest_match['neutered']}")
    print(f"Housebroken: {closest_match['housebroken']}")
    print(f"Likes People: {closest_match['likes_people']}")
    print(f"Likes Children: {closest_match['likes_children']}")
    print(f"Gets Along with Males: {closest_match['get_along_males']}")
    print(f"Gets Along with Females: {closest_match['get_along_females']}")
    print(f"Gets Along with Cats: {closest_match['get_along_cats']}")
    print(f"Best Kept In: {closest_match['keep_in']}")
    #print(f"Similarity Score: {closest_match['similarity']:.2f}")
    print("----------------------------")

    # display other matches in a table
    print("\nOther Matches:")
    print(sorted_dogs.iloc[1:5][['name', 'age', 'sex', 'breed']].to_string(index=False))

    print("\nYou can find more details about your selected dog on our website.\nYou can visit us any day between 9AM - 5PM to meet your new furry friend.\nWe can't wait to help you find the perfect companion!")


# run the function to get user preferences
user_preferences = get_user_preferences()

# print the final list of user's preferences if they have provided any
if user_preferences:
    print("\nYour preferences are:")
    for attribute, value in user_preferences.items():
        print(f"{attribute}: {value}")
else:
    print("\nNo preferences were provided.")


# run the function to recommend a dog to the user
recommend_dog(user_preferences, shelter_dogs)


Welcome to PawPal Pet Adoption Center. 
We are here to help you find your new furry friend! 

Help us understand which attribute is most important to you? (or press Enter to finish):
1: age
2: sex
3: breed
4: color
5: coat
6: size
7: neutered
8: housebroken
9: likes_people
10: likes_children
11: get_along_males
12: get_along_females
13: get_along_cats
14: keep_in
You have selected: age with preference: puppy

Help us understand which attribute is most important to you? (or press Enter to finish):
1: age
2: sex
3: breed
4: color
5: coat
6: size
7: neutered
8: housebroken
9: likes_people
10: likes_children
11: get_along_males
12: get_along_females
13: get_along_cats
14: keep_in
You have selected: sex with preference: male

Help us understand which attribute is most important to you? (or press Enter to finish):
1: age
2: sex
3: breed
4: color
5: coat
6: size
7: neutered
8: housebroken
9: likes_people
10: likes_children
11: get_along_males
12: get_along_females
13: get_along_cats
14: keep_